<table class="table table-bordered">
    <tr>
        <th style="text-align:center; width:25%"><img src='https://www.np.edu.sg/PublishingImages/Pages/default/odp/ICT.jpg' style="width: 250px; height: 125px; "></th>
        <th style="text-align:center;"><h1>Deep Learning</h1><h2>Practical 2b - Predicting House Prices</h2><h3>AY2020/21 Semester</h3></th>
    </tr>
</table>

In [1]:
from tensorflow import keras
print('keras: ', keras.__version__)

keras:  2.2.4-tf


In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Objectives
After completing this practical exercise, students should be able to:
1. [Build a neural network model to predict house prices](#demo)
2. [Exercise- tuning several model parameters](#exc)

## 1. Predicting house prices (a regression example) <a id='demo' />

### 1.1 The Boston Housing Price dataset

We will be attempting to predict the median price of homes in a given Boston suburb in the mid-1970s, given a few data points about the 
suburb at the time, such as the crime rate, the local property tax rate, etc.

The dataset has only 506 samples, split between 404 training samples and 102 test samples. Let's take a look at the data:

In [2]:
from tensorflow.keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [4]:
print(train_data[100])

[6.1290e-02 2.0000e+01 3.3300e+00 1.0000e+00 4.4290e-01 7.6450e+00
 4.9700e+01 5.2119e+00 5.0000e+00 2.1600e+02 1.4900e+01 3.7707e+02
 3.0100e+00]


In [5]:
test_data.shape

(102, 13)


As you can see, we have 404 training samples and 102 test samples. The data comprises 13 features (details are shown below) and each "feature" in the input data (e.g. the crime rate is a feature) has a different scale. For instance some values are proportions, which take a values between 0 and 1, others take values between 1 and 12, others between 0 and 100...

1. Per capita crime rate.
2. Proportion of residential land zoned for lots over 25,000 square feet.
3. Proportion of non-retail business acres per town.
4. Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
5. Nitric oxides concentration (parts per 10 million).
6. Average number of rooms per dwelling.
7. Proportion of owner-occupied units built prior to 1940.
8. Weighted distances to five Boston employment centres.
9. Index of accessibility to radial highways.
10. Full-value property-tax rate per $10,000.
11. Pupil-teacher ratio by town.
12. 1000 * (Bk - 0.63) ** 2 where Bk is the proportion of Black people by town.
13. % lower status of the population.

The targets are the median values of owner-occupied homes, in thousands of dollars:

In [6]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

The prices are typically between $10,000  -  $50,000. If that sounds cheap, remember this was the mid-1970s, and these prices are not inflation-adjusted.

### 1.2 Preparing the data

It would be problematic to feed into a neural network values that all take wildly different ranges. The network might be able to 
automatically adapt to such heterogeneous data, but it would definitely make learning more difficult. A widespread best practice to deal with such data is to do feature-wise normalization: for each feature in the input data (a column in the input data matrix), we will subtract the mean of the feature and divide by the standard deviation, so that the feature will be centered around 0 and will have a unit standard deviation. This is easily done in Numpy:

In [7]:
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

Note that the quantities that we use for normalizing the test data have been computed using the training data. We should never use in our workflow any quantity computed on the test data, even for something as simple as data normalization.

In [8]:
print(train_data[100])

[-0.39914449  0.35890566 -1.14281587  3.89358447 -0.97702129  1.9437178
 -0.69198737  0.72576261 -0.51114231 -1.1428069  -1.62718308  0.23710757
 -1.34300395]


### 1.3 Building our network


Because so few samples are available, we will be using a very small network with two 
hidden layers, each with 64 units. In general, the less training data you have, the worse overfitting will be, and using 
a small network is one way to mitigate overfitting.

In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(64, activation='relu',
                        input_shape=(train_data.shape[1],)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='mse', metrics=['mae'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                896       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


Our network ends with a single unit, and no activation (i.e. linear layer). This is a typical setup for scalar regression. Because the last layer is purely linear, the network is free to learn to predict values in any range.

Note that we are compiling the network with the `mse` loss function -- Mean Squared Error, the square of the difference between the predictions and the targets, a widely used loss function for regression problems.

We are also monitoring a new metric during training: `mae`. This stands for Mean Absolute Error. It is simply the absolute value of the difference between the predictions and the targets. For instance, a MAE of 0.5 on this problem would mean that our predictions are off by 
\$500 on average.

In [10]:
model.fit(train_data, train_targets, validation_split =0.2, epochs = 200, batch_size = 1)

Train on 323 samples, validate on 81 samples
Epoch 1/200
  1/323 [..............................] - ETA: 4:08

InternalError:  Blas GEMM launch failed : a.shape=(1, 13), b.shape=(13, 64), m=1, n=64, k=13
	 [[node sequential_1/dense/MatMul (defined at <ipython-input-10-25e9c7968c01>:1) ]] [Op:__inference_distributed_function_777]

Function call stack:
distributed_function


In [ ]:
As you can see, the model very quickly overfits to the training data, so we should stop it before it overfits. Now the question is when to stop? We will use K-fold validation to figure out in the next secsion.  

Because you'll need to instantiate the same model multiple times, you use a function to construct it.

In [16]:
from tensorflow.keras import models
from tensorflow.keras import layers

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

### 1.4 Validating our approach using K-fold validation

To evaluate our network while we keep adjusting its parameters (e.g. the number of epochs), we use K-fold cross-validation because we have so few data points. It splits the data into K partitions, then instantiating K identical models, and training each one on K-1 partitions while evaluating on the remaining partition. The validation score for the model used would then be the average of the K validation scores obtained.

In [17]:
from tensorflow.keras import backend as K
# Some memory clean-up
K.clear_session()

In [ ]:
import numpy as np

k = 5
num_val_samples = len(train_data) // k
num_epochs = 120
all_scores = []

all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=1)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)

We can then compute the average of the per-epoch MAE scores for all folds:

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

Let's plot this:

In [11]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(range(1, len(average_mae_history) + 1)[5:], average_mae_history[5:])
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

NameError: name 'average_mae_history' is not defined

It seems that validation MAE stops improving significantly after 50 epochs. We can now train a final "production" model on all of the training data, with the best parameters, then look at its performance on the test data:

In [ ]:
# Get a fresh, compiled model.
model = build_model()
# Train it on the entirety of the data.
history = model.fit(train_data, train_targets, validation_split =0.2,
          epochs=50, batch_size=1, verbose=1)


In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
mae = history.history['mae']
val_mae = history.history['val_mae']

epochs = range(1, len(mae) + 1)

plt.plot(epochs[10:], mae[10:], 'r', label='mean_absolute_error')
plt.plot(epochs[10:], val_mae[10:], 'b', label='val_mean_absolute_error')
plt.title('Training and validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets, verbose=2)
test_mae_score

## 2. Exercise - tuning model parameters <a id='exc' />
Please train the above model in the below two scenerios: make the changes on the indicated training configurations (the rest no change). Train both models for 120 epochs. 

**Scenerio A**:
* change the batch size from 1 to 128

**Scenerio B**:
* change the learning rate (`optimizers.RMSprop(lr=0.001)`) from 0.001 to 0.0002

Observe the training and validation MAE curves for both scenerios.

Provide your codes & observations in the below boxes.

### 2.1 Scenerio A

In [ ]:
# Task 1: Build the model and no changes


In [ ]:
# Task 2: Compile and Train the model for 200 epochs. Change the batch size from 1 to 128


In [ ]:
#Task 3: Plot the MAE (train & test) curves


In [ ]:
#Task 4: Comment on your model


### 2.2 Scenerio B

In [ ]:
#Task 1: Build the model


In [ ]:
#Task 2: Compile and Fit the model. Change the learning rate from 0.001 to 0.0002


In [ ]:
#Task 3: Plot the MAE (train & test) curves


In [ ]:
#Task 4: Comment on your model
